In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord

In [ ]:
# https://sites.google.com/usfca.edu/neuralens/publications/lens-candidates-storfer-et-al-2022
huang20 = pd.read_csv('/home/walml/repos/galaxy-datasets/data/external_data/neuralens/candidates_H20.csv')
huang21 = pd.read_csv('/home/walml/repos/galaxy-datasets/data/external_data/neuralens/candidates_H21.csv')
storfer22 = pd.read_csv('/home/walml/repos/galaxy-datasets/data/external_data/neuralens/storfer22_full.csv')

In [ ]:
huang20.columns.values

Tidy up the columns

In [ ]:
huang20_renamer = {
    'Name': 'name', 'Probability': 'probability', 'Spec': 'spec_z', 'Photo-z': 'phot_z', 'g mag': 'mag_g', 'r mag': 'mag_r', 'z mag': 'mag_z',
}

huang20 = huang20.rename(columns=huang20_renamer)
huang20 = huang20[huang20_renamer.values()]
huang20

In [ ]:
huang21_renamer = huang20_renamer.copy()
huang21_renamer.update({'Score': 'score', 'Score Diff': 'score_diff'})
huang21 = huang21.rename(columns=huang21_renamer)

In [ ]:
huang21 = huang21[huang21_renamer.values()]

In [ ]:
# storfer doesn't need renaming, only subset

storfer22 = storfer22.rename(columns={'phot-z': 'phot_z', 'spec-z': 'spec_z'})

storfer22 = storfer22[['name', 'ra', 'dec', 'grade', 'probability', 'spec_z', 'phot_z', 'mag_g', 'mag_r', 'mag_z']]
storfer22

Huang 20 and Huang 21 don't include RA/Dec cols, so parse these out from the DESI names

In [ ]:
# def desi_name_to_coords(name):
#     ra_str = name[5:13]  # DESI- prefix always, RA is 0 to 360
#     dec_str = name[13:]  # sometimes -, sometimes+, Dec is -90 to +90
#     # print(ra_str, dec_str)
#     return SkyCoord(ra=ra_str, dec=dec_str, frame='icrs', unit='deg')

# desi_name_to_coords('DESI-351.4891-00.8741')


def desi_names_to_coords(names: pd.Series):
    # parse RA/Dec from DESI name
    ra_strs = names.apply(lambda x: x[5:13]).values  # DESI- prefix always, RA is 0 to 360
    dec_strs = names.apply(lambda x: x[13:]).values # sometimes -, sometimes+, Dec is -90 to +90
    skycoord = SkyCoord(ra=ra_strs, dec=dec_strs, frame='icrs', unit='deg')
    return skycoord.ra.deg, skycoord.dec.deg

huang20_ra, huang20_dec = desi_names_to_coords(huang20['name'])
huang20['ra'] = huang20_ra
huang20['dec'] = huang20_dec

huang21_ra, huang21_dec = desi_names_to_coords(huang21['name'])
huang21['ra'] = huang21_ra
huang21['dec'] = huang21_dec

### Labels: grades, scores, oh my

Huang 20 doesn't have explicit grades listed on the sheet, but:

"[In the three tables below](https://sites.google.com/usfca.edu/neuralens/publications/lens-candidates-huang-et-al-2020), the 342 candidate systems from Huang et al. 2020 are grouped by Grade, each Grade is arranged in ascending RA."

So let's encode that ourselves.

In [ ]:
huang20_grades = ['A'] * 60 + ['B'] * 106 + ['C'] * 176
huang20['grade'] = huang20_grades

Huang 21 uses scores, from 2 to 4 apparently.
<!-- https://arxiv.org/pdf/2005.04730.pdf -->

"
Co-authors S.B., A.G.,
A.P., V.R., C.S., W.S., and R.V. make the “first pass” selections, according to these criteria, erring
on the generous side

For the “second
pass”, co-authors X.H. and A.D. examine all “first pass” selections and assign an integer score
between 1 and 4. These two scores are averaged. We assign a letter grade according to the average,
using the following criteria.

≥ 3.5: Grade A. We have a high level of confidence of these candidates...

= 3.0: Grade B. They have similar characteristics as the Grade A’s. For the cutout images
where there appear to be giant arcs they tend to be fainter than those for the Grade A’s.

= 2.5 or 2.0: Grade C. They generally have features that are even fainter and/or smaller than
what is typical for Grade B candidates, but that are nevertheless suggestive of lensed arclets.
Counter images are often not present or indiscernible.
"

In [ ]:
huang21['score'].value_counts()

Storfer returns to the same grade system as Huang20, but adds a D grade (not described in the paper, but presumably unlikely or rejected lenses)

In [ ]:
storfer22['grade'].value_counts()

Only H21 uses score/score delta, so let's simplify and stick with the grade system.

In [ ]:
def score_to_grade(score):
    if score >= 3.5:
        return 'A'
    elif score >= 3.0:
        return 'B'
    else: 
        return 'C'

huang21['grade'] = huang21['score'].apply(score_to_grade)

In [ ]:
huang20['grade'].value_counts(normalize=True).sort_index()

In [ ]:
huang21['grade'].value_counts(normalize=True).sort_index()

In [ ]:
# storfer22['grade'].value_counts(normalize=True).sort_index()

"Grade D systems are not counted as candidates in this paper, but we have
included them on our project website (URL provided in §4.2)."

In [ ]:
storfer22 = storfer22[storfer22['grade'] != 'D']
storfer22['grade'].value_counts(normalize=True).sort_index()
# storfer has far fewer grade A's than H20 and H21, and H20 has more B vs C than H21 and storfer. Unclear if rating inconsistency or due to previous mining.

### Join the candidate lists

In [ ]:
huang20.head()

In [ ]:
huang21.head()

In [ ]:
storfer22.head()

In [ ]:
huang20['label_origin'] = 'huang20'
huang21['label_origin'] = 'huang21'
storfer22['label_origin'] = 'storfer22'

df = pd.concat([huang20, huang21, storfer22], axis=0)
del df['score']
del df['score_diff']

df['phot_z_mean'] = df['phot_z'].apply(lambda x: float(x.split('+')[0].split('±')[0]) if type(x) == str else np.nan)

def get_best_z(galaxy):
    if not pd.isna(galaxy['spec_z']):
        return galaxy['spec_z']
    return galaxy['phot_z_mean']

df['best_z'] = df.apply(get_best_z, axis=1)

df.sample(10)

In [ ]:
df['phot_z']

In [ ]:
fig, axes = plt.subplots(nrows=3)

# df_to_show = df
# bins = 30

bins = 10
df_to_show = df.query('grade == "A"')

ax = axes[0]
ax.hist(df_to_show['mag_r'], bins=bins)
ax.set_xlabel('mag_r')


ax = axes[1]
ax.hist(df_to_show['best_z'], bins=bins)
ax.set_xlabel('best_z')
ax.set_xlim([0., 1.])

ax = axes[2]
ax.hist(df_to_show['spec_z'], bins=bins)
ax.set_xlabel('spec_z')
ax.set_xlim([0., 1.])

fig.tight_layout()

In [ ]:
# crossmatch to DESI LS DR8, to my images

In [ ]:
# TODO crossmatch to DR9 for Stein's images?

In [ ]:
df['grade'].value_counts()

In [ ]:
df.to_csv('/home/walml/repos/galaxy-datasets/data/external_data/neuralens/neuralens_combined.csv', index=False)